https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [1]:
import scipy.stats as sci_stats
from pathlib import Path
import numpy as np
import json

In [2]:
XAI_metrics_folder = Path("../XAI_metrics/metrics")

In [3]:
with open(XAI_metrics_folder / "final_gb_grad_cam_map_metrics.json") as f:
    data = json.load(f)

In [4]:
grad_cam_final = np.array(data["values"], dtype=float)
grad_cam_final.shape

(49595,)

In [5]:
with open(XAI_metrics_folder / "final_grad_cam_map_metrics.json") as f:
    data = json.load(f)

In [6]:
grad_cam_initial = np.array(data["values"], dtype=float)

In [12]:
ttest = sci_stats.ttest_rel(grad_cam_final, grad_cam_initial, nan_policy='omit')

In [13]:
ttest.pvalue

0.0

In [17]:
np.nanmean(grad_cam_initial), np.nanmean(grad_cam_final), np.nanstd(grad_cam_initial), np.nanstd( grad_cam_final), len(grad_cam_initial), len(grad_cam_final)

(0.22737722966876167,
 0.27364923411374886,
 0.1146301005987649,
 0.10917829100512624,
 49595,
 49595)